<a href="https://colab.research.google.com/github/haphananhtuan/chemical_classification/blob/master/Convert_Tabular_Data_to_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create folder and Load dataset

In [0]:
!mkdir data_sample

In [0]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import tensorflow as tf


In [0]:
data_path = "/content/data_sample/"
data = pd.DataFrame()


for filename in os.listdir(data_path):
    absorbance = pd.read_csv(data_path + filename)["Absorbance (AU)"]
    absorbance.at[absorbance.size] = filename[:3]
    data = data.append(absorbance, ignore_index=True)

Remove class which only have few samples

In [0]:
classes = ['Cbs', 'Mel', 'Cel', 'Gua', 'P-S']
data = data[~data[228].isin(classes)]


target = data[228].value_counts()
target = target[target < 40].reset_index()['index'].values # get the classes that have least data points
data_augmented = data.copy()

# simple data augmentation by adding gaussian noise to the mean of each feature of a sample
num_adding = 40
for label in target:
    label_mean = data_augmented[data_augmented[228] == label].drop(228, axis=1).mean().values
    for _ in range(num_adding):
        sample = pd.Series(label_mean)
        sample += np.random.randn(sample.size) * 0.01
        sample.at[sample.size] = label
        data_augmented = data_augmented.append(sample, ignore_index=True)
        
X = data_augmented.drop(228, axis=1).values
Y = data_augmented[228].values

In [0]:
print(X.shape)

(1214, 228)


In [0]:
fig = plt.figure(figsize=(10, 5))

for 